In [1]:
# SISTEMA Y UTILIDADES
import os
import random
import numpy as np
from collections import Counter
from pathlib import Path

# VISUALIZACIÓN Y EVALUACIÓN
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
import seaborn as sns

# TORCH
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms

# SOLUCIÓN PARA ARCHIVOS CORRUPTOS
from PIL import UnidentifiedImageError
from torchvision.datasets import ImageFolder

# Clase personalizada para evitar errores al cargar imágenes corruptas
class SafeImageFolder(ImageFolder):
    def __getitem__(self, index):
        try:
            return super().__getitem__(index)
        except (UnidentifiedImageError, OSError):
            new_index = (index + 1) % len(self)
            return self.__getitem__(new_index)

In [2]:
# Ruta al dataset
DATA_DIR = Path("/Users/alvarosanchez/Downloads/MURA-v1.1")  # Cambia esto a "data/MURA-v1.1" para usar el dataset completo

# Transformaciones para entrenamiento con augmentación
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Transformaciones para validación: sin augmentación
transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
])

# Carga segura de datasets
train_dataset = SafeImageFolder(DATA_DIR / "train", transform=transform_train)
valid_dataset = SafeImageFolder(DATA_DIR / "valid", transform=transform_val)

# DataLoaders
BATCH_SIZE = 32
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Verificamos las clases
print("Clases:", train_dataset.classes)

Clases: ['XR_ELBOW', 'XR_FINGER', 'XR_FOREARM', 'XR_HAND', 'XR_HUMERUS', 'XR_SHOULDER', 'XR_WRIST']


RandomHorizontalFlip, Rotation y ColorJitter simulan variaciones reales que podrías encontrar en radiografías (posiciones del brazo, luz, etc.).

El uso de Normalize es importante porque los modelos preentrenados en ImageNet esperan imágenes normalizadas con esas medias y desviaciones estándar.

SafeImageFolder es clave para evitar errores con imágenes corruptas (frecuentes en MURA)

In [3]:
# Cargamos modelo preentrenado con pesos de ImageNet
model = models.resnet18(weights="IMAGENET1K_V1")

# Congelamos TODAS las capas al principio
for param in model.parameters():
    param.requires_grad = False

# Descongelamos la última capa convolucional: layer4
for param in model.layer4.parameters():
    param.requires_grad = True

# También descongelamos la capa totalmente conectada (fc)
for param in model.fc.parameters():
    param.requires_grad = True

# Ajustamos la capa de salida para clasificación binaria
in_features = model.fc.in_features
model.fc = nn.Linear(in_features, 2)  # 2 clases: 'negative', 'positive'

# Enviamos el modelo al dispositivo adecuado (GPU o Apple M3)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
model = model.to(DEVICE)

Este modelo se basa en ResNet18, una red convolucional profunda entrenada sobre ImageNet. 

En lugar de entrenar todo desde cero, aprovechamos su conocimiento general de imágenes (bordes, formas, texturas), 

pero le permitimos reajustar su parte final (layer4) para que aprenda las particularidades de las radiografías musculoesqueléticas. 

Esto se conoce como fine-tuning parcial.

La última capa (fc) también se entrena desde cero para adaptarse a nuestra tarea de clasificación binaria: positive o negative.

In [10]:
# Conteo de clases reales (binary: 0 = negative, 1 = positive)
labels = [label for _, label in train_dataset.samples]
class_counts = Counter(labels)
print("Distribución de clases:", class_counts)

# Cálculo correcto para 2 clases
total = sum(class_counts.values())
class_weights = [total / class_counts[i] for i in range(2)]
weights = torch.tensor(class_weights, dtype=torch.float).to(DEVICE)

# Función de pérdida con pesos binarios
criterion = nn.CrossEntropyLoss()

Distribución de clases: Counter({6: 9752, 5: 8379, 3: 5543, 1: 5106, 0: 4931, 2: 1825, 4: 1272})


Para evitar que el modelo favorezca la clase mayoritaria, calculamos pesos de clase basados en su frecuencia inversa. 

De este modo, cometer errores en la clase menos representada penaliza más durante el entrenamiento, 

ayudando a equilibrar la precisión y el recall.

In [11]:
# Hiperparámetros
EPOCHS = 25
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5
PATIENCE = 5  # early stopping

# Optimizador y scheduler
optimizer = optim.Adam(model.parameters(), lr=1e-4)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5,
                                                 patience=2, verbose=True)

# Entrenamiento con early stopping
train_losses, val_losses = [], []
best_val_loss = float('inf')
epochs_no_improve = 0

for epoch in range(EPOCHS):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    avg_train_loss = running_loss / len(train_loader)
    train_losses.append(avg_train_loss)

    # Validación
    model.eval()
    val_loss = 0.0
    y_true, y_pred = [], []

    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            y_true.extend(labels.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    avg_val_loss = val_loss / len(valid_loader)
    val_losses.append(avg_val_loss)
    scheduler.step(avg_val_loss)

    # Métricas
    f1 = f1_score(y_true, y_pred, average='macro')
    acc = accuracy_score(y_true, y_pred)

    print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | "
          f"Val Loss: {avg_val_loss:.4f} | Acc: {acc:.4f} | F1: {f1:.4f}")

    # Guardado del mejor modelo
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        os.makedirs("src/models", exist_ok=True)
        torch.save(model.state_dict(), "/Users/alvarosanchez/ONLINE_DS_THEBRIDGE_ALVAROSMMS-1/ML_Clasificacion_Radiografias_Muscoesqueleticas/src/models/resnet18_finetuned.pt")
        print("Mejor modelo guardado.")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print("Early stopping activado.")
            break

/opt/homebrew/lib/python3.11/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1 | Train Loss: 0.0019 | Val Loss: 0.0018 | Acc: 0.2893 | F1: 0.1282
Mejor modelo guardado.
Epoch 2 | Train Loss: 0.0011 | Val Loss: 0.0038 | Acc: 0.2890 | F1: 0.1282
Epoch 3 | Train Loss: 0.0007 | Val Loss: 0.0006 | Acc: 0.2893 | F1: 0.1284
Mejor modelo guardado.
Epoch 4 | Train Loss: 0.0007 | Val Loss: 0.0009 | Acc: 0.2893 | F1: 0.1286
Epoch 5 | Train Loss: 0.0009 | Val Loss: 0.0006 | Acc: 0.2896 | F1: 0.1283
Epoch 6 | Train Loss: 0.0005 | Val Loss: 0.0013 | Acc: 0.2890 | F1: 0.1281
Epoch 7 | Train Loss: 0.0008 | Val Loss: 0.0003 | Acc: 0.2896 | F1: 0.1289
Mejor modelo guardado.
Epoch 8 | Train Loss: 0.0005 | Val Loss: 0.0005 | Acc: 0.2893 | F1: 0.1284
Epoch 9 | Train Loss: 0.0003 | Val Loss: 0.0012 | Acc: 0.2893 | F1: 0.1286
Epoch 10 | Train Loss: 0.0001 | Val Loss: 0.0006 | Acc: 0.2893 | F1: 0.1283
Epoch 11 | Train Loss: 0.0003 | Val Loss: 0.0007 | Acc: 0.2893 | F1: 0.1282
Epoch 12 | Train Loss: 0.0002 | Val Loss: 0.0017 | Acc: 0.2893 | F1: 0.1287
Early stopping activado.
